### Notes in LLM Google doc

In [5]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]='0,1'

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import textwrap
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

device = "cuda" if torch.cuda.is_available() else "cpu"
#Need only 1 GPU if loading 8-bit model
print(device)

print("Using %d GPUs" %torch.cuda.device_count())

cuda
Using 4 GPUs


In [2]:
model_name = "eachadea/vicuna-13b-1.1"
tokenizer_path = "./tokenizer/"

#Create a local tokenizer copy the first time
if os.path.isdir(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
else:
    tokenizer = AutoTokenizer.from_pretrained("model_name")
    os.mkdir(tokenizer_path)
    tokenizer.save_pretrained(tokenizer_path)


In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")#, load_in_8bit=True)
#model = AutoModelForCausalLM.from_pretrained(model_name).to(device) -- tries to put everything on 1 GPU

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [36]:
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=256,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

In [65]:
#question = "How can you use coherent imaging in materials science?"
#question = "What is the capital of England?"
question = "Hi, I am Mathew"
a = pipe(question)

In [66]:
textwrap.wrap(a[0]['generated_text'])

['Hi, I am Mathew from Dubai. I have a 2016 Ford Ranger Wildtrak and I',
 'would like to know if the OEM front bumper can be replaced with an',
 'aftermarket one? Also what are the regulations in UAE regarding such',
 'modifications? Thank you!']

In [68]:
local_llm = HuggingFacePipeline(pipeline=pipe)

In [69]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
{instruction}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["instruction"])

In [70]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

b=llm_chain.run(question)

In [73]:
textwrap.wrap(b)

[' Hello Mathew! How can I assist you today?']